**Imports**

In [219]:
import yfinance as yf
from datetime import datetime
import pandas as pd
from typing import Tuple
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np
import math
import QuantLib as ql

**Define price fetcher**

In [220]:
def get_spot_price(ticker):
    """
    Fetches the current spot price for a stock ticker. Falls back to the most recent
    close if a live price is not available.

    Parameters
    ----------
    ticker : str
        Stock ticker symbol (e.g., 'AAPL').

    Returns
    -------
    float or None
        Spot price (live if available, else last close). Returns None if unavailable.
    """

    try:
        tk = yf.Ticker(ticker)
        
        # Attempt to fetch live price
        live_price = tk.fast_info.get("last_price", None)
        if live_price and live_price > 0:
            return live_price

        # Fallback: most recent close
        hist = tk.history(period="1d")
        if not hist.empty:
            fallback_price = hist["Close"].iloc[-1]
            print(f"[{ticker}] Live price unavailable — using last close: {fallback_price:.2f}")
            return fallback_price

        print(f"[{ticker}] No live or historical data available.")
        return None

    except Exception as e:
        print(f"[{ticker}] Spot price fetch failed: {e}")
        return None

**Define Option Chain Fetcher**

In [221]:
def get_option_chains_all(ticker: str,
                                  max_workers: int = 8) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Fetches option chains (calls and puts) for every available expiry of a given ticker,
    performing API requests in parallel to reduce total fetch time.

    Parameters
    ----------
    ticker : str
        Stock ticker symbol (e.g., 'AAPL').
    max_workers : int, optional
        Maximum number of threads to use for concurrent fetching (default is 8).

    Returns
    -------
    Tuple[pd.DataFrame, pd.DataFrame]
        - calls_df: DataFrame containing all calls across expiries, with added columns:
            * 'option_type' = 'call'
            * 'expiration'  = expiry date string 'YYYY-MM-DD'
            * 'TTM'         = time to maturity in years
        - puts_df: DataFrame containing all puts with the same added columns.
    """
    stock = yf.Ticker(ticker)
    expiries = stock.options  # list of expiry date strings
    today = datetime.now().date()

    calls_accum = []
    puts_accum  = []

    def fetch_chain(expiry: str):
        """Fetch calls/puts for a single expiry and return (expiry, calls_df, puts_df)."""
        try:
            chain = stock.option_chain(expiry)
            calls = chain.calls.copy()
            puts  = chain.puts.copy()
        except Exception as e:
            # Return None on error so we can skip later
            return expiry, None, None

        # Tag each row with type and expiration
        calls['option_type']  = 'call'
        puts ['option_type']  = 'put'
        calls['expiration']   = expiry
        puts ['expiration']   = expiry

        # Compute time-to-maturity once
        exp_date = datetime.strptime(expiry, "%Y-%m-%d").date()
        ttm = max((exp_date - today).days / 365.0, 0.0)
        calls['TTM'] = ttm
        puts ['TTM'] = ttm

        return expiry, calls, puts

    # Fetch in parallel
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(fetch_chain, exp) for exp in expiries]
        for future in as_completed(futures):
            expiry, calls_df, puts_df = future.result()
            if calls_df is not None and not calls_df.empty:
                calls_accum.append(calls_df)
            if puts_df  is not None and not puts_df.empty:
                puts_accum.append(puts_df)

    # Concatenate results
    all_calls = pd.concat(calls_accum, ignore_index=True) if calls_accum else pd.DataFrame()
    all_puts  = pd.concat(puts_accum,  ignore_index=True) if puts_accum  else pd.DataFrame()

    # Fetch dividend yield for the company
    dividendYield = stock.info.get("dividendYield")/100 # percentages on decimal basis
    all_calls["dividendYield"] = dividendYield
    all_puts["dividendYield"] = dividendYield

    all_calls["ticker"] = ticker
    all_puts["ticker"] = ticker

    spot_price = get_spot_price(ticker)
    all_calls["spot_price"] = spot_price
    all_puts["spot_price"] = spot_price

    return all_calls, all_puts


Get option chain and dividend yield

In [222]:
calls, puts = get_option_chains_all("JPM")

[JPM] Live price unavailable — using last close: 314.21


In [223]:
calls

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,option_type,expiration,TTM,dividendYield,ticker,spot_price
0,JPM251128C00180000,2025-11-03 18:02:15+00:00,180.0,130.43,132.70,136.15,0.00,0.000000,20.0,10,0.989258,True,REGULAR,USD,call,2025-11-28,0.057534,0.0191,JPM,314.209991
1,JPM251128C00185000,2025-10-22 16:30:49+00:00,185.0,107.25,127.70,131.15,0.00,0.000000,NaN,1,0.945313,True,REGULAR,USD,call,2025-11-28,0.057534,0.0191,JPM,314.209991
2,JPM251128C00225000,2025-10-10 18:50:55+00:00,225.0,79.80,87.85,91.40,0.00,0.000000,NaN,1,0.690433,True,REGULAR,USD,call,2025-11-28,0.057534,0.0191,JPM,314.209991
3,JPM251128C00230000,2025-10-14 13:35:01+00:00,230.0,71.05,82.85,86.30,0.00,0.000000,NaN,0,0.637699,True,REGULAR,USD,call,2025-11-28,0.057534,0.0191,JPM,314.209991
4,JPM251128C00250000,2025-10-20 17:42:50+00:00,250.0,54.50,63.00,66.45,0.00,0.000000,2.0,2,0.518560,True,REGULAR,USD,call,2025-11-28,0.057534,0.0191,JPM,314.209991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
879,JPM280121C00430000,2025-10-21 19:58:31+00:00,430.0,9.85,13.70,15.10,0.00,0.000000,1.0,2,0.250496,False,REGULAR,USD,call,2028-01-21,2.205479,0.0191,JPM,314.209991
880,JPM280121C00440000,2025-10-31 16:59:23+00:00,440.0,11.60,10.65,14.45,0.00,0.000000,4.0,14,0.255989,False,REGULAR,USD,call,2028-01-21,2.205479,0.0191,JPM,314.209991
881,JPM280121C00450000,2025-10-30 17:44:38+00:00,450.0,9.90,11.00,12.90,0.00,0.000000,4.0,17,0.254539,False,REGULAR,USD,call,2028-01-21,2.205479,0.0191,JPM,314.209991
882,JPM280121C00460000,2025-10-22 16:30:37+00:00,460.0,6.03,9.95,11.55,0.00,0.000000,2.0,6,0.253532,False,REGULAR,USD,call,2028-01-21,2.205479,0.0191,JPM,314.209991


In [224]:
puts

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,option_type,expiration,TTM,dividendYield,ticker,spot_price
0,JPM251128P00175000,2025-10-22 19:47:28+00:00,175.0,0.03,0.00,2.13,0.000000,0.000000,NaN,11,1.321781,False,REGULAR,USD,put,2025-11-28,0.057534,0.0191,JPM,314.209991
1,JPM251128P00220000,2025-10-17 14:28:32+00:00,220.0,0.39,0.00,2.14,0.000000,0.000000,3.0,2,0.864747,False,REGULAR,USD,put,2025-11-28,0.057534,0.0191,JPM,314.209991
2,JPM251128P00235000,2025-10-23 18:37:04+00:00,235.0,0.48,0.00,2.16,0.000000,0.000000,NaN,13,0.730716,False,REGULAR,USD,put,2025-11-28,0.057534,0.0191,JPM,314.209991
3,JPM251128P00240000,2025-10-17 19:24:44+00:00,240.0,0.60,0.00,2.17,0.000000,0.000000,10.0,11,0.687747,False,REGULAR,USD,put,2025-11-28,0.057534,0.0191,JPM,314.209991
4,JPM251128P00245000,2025-10-27 19:48:25+00:00,245.0,0.29,0.00,2.06,0.000000,0.000000,10.0,82,0.637211,False,REGULAR,USD,put,2025-11-28,0.057534,0.0191,JPM,314.209991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,JPM280121P00400000,2025-11-07 19:10:43+00:00,400.0,95.10,92.00,94.80,-3.849998,-3.890852,10.0,3,0.177514,True,REGULAR,USD,put,2028-01-21,2.205479,0.0191,JPM,314.209991
779,JPM280121P00410000,2025-09-29 16:42:36+00:00,410.0,99.85,100.00,103.50,0.000000,0.000000,NaN,1,0.178094,True,REGULAR,USD,put,2028-01-21,2.205479,0.0191,JPM,314.209991
780,JPM280121P00420000,2025-10-09 18:48:52+00:00,420.0,116.05,108.75,112.00,0.000000,0.000000,NaN,2,0.175408,True,REGULAR,USD,put,2028-01-21,2.205479,0.0191,JPM,314.209991
781,JPM280121P00460000,2025-09-30 15:35:29+00:00,460.0,149.81,143.50,148.00,0.000000,0.000000,NaN,0,0.163644,True,REGULAR,USD,put,2028-01-21,2.205479,0.0191,JPM,314.209991


In [225]:
calls.describe()

,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,TTM,dividendYield,spot_price
count,884.000000,884.000000,884.000000,884.000000,884.000000,884.000000,837.000000,884.000000,884.000000,884.000000,884.0000,884.000000
mean,279.736991,59.886765,61.534706,63.706968,0.009514,1.305206,65.540024,436.348416,0.433174,0.635917,0.0191,314.209991
std,94.121347,59.276993,61.384402,62.465660,1.283287,34.770268,362.538300,993.660580,0.387665,0.652016,0.0000,0.000000
min,65.000000,0.010000,0.000000,0.000000,-5.450005,-98.461540,1.000000,0.000000,0.000010,0.000000,0.0191,314.209991
25%,200.000000,7.097500,6.487500,7.737500,0.000000,0.000000,2.000000,12.000000,0.259918,0.115068,0.0191,314.209991
50%,287.500000,38.625000,38.475000,40.750000,0.000000,0.000000,4.000000,72.000000,0.327292,0.364384,0.0191,314.209991
75%,350.000000,107.340000,111.487500,114.225000,0.000000,0.000000,20.000000,349.750000,0.482732,1.112329,0.0191,314.209991
max,470.000000,240.200000,242.750000,246.100000,13.110001,800.000240,6744.000000,8789.000000,6.646486,2.205479,0.0191,314.209991


Initial Dataset Clean

In [226]:
def initial_dataset_clean(dataset : pd.DataFrame):
    # Removing columns that will not be used for training or filtering
    #cols_to_drop = ["contractSymbol", "lastTradeDate", "change", "percentChange", "expiration", "inTheMoney"]
    #dataset.drop(columns=[c for c in cols_to_drop if c in dataset.columns], inplace=True)

    # Removing NA values for stale options
    dataset.dropna(axis=0,subset=["volume"], inplace=True)

    
    return dataset

In [227]:
calls_clean = initial_dataset_clean(calls)
calls_clean

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,option_type,expiration,TTM,dividendYield,ticker,spot_price
0,JPM251128C00180000,2025-11-03 18:02:15+00:00,180.0,130.43,132.70,136.15,0.00,0.000000,20.0,10,0.989258,True,REGULAR,USD,call,2025-11-28,0.057534,0.0191,JPM,314.209991
4,JPM251128C00250000,2025-10-20 17:42:50+00:00,250.0,54.50,63.00,66.45,0.00,0.000000,2.0,2,0.518560,True,REGULAR,USD,call,2025-11-28,0.057534,0.0191,JPM,314.209991
6,JPM251128C00270000,2025-10-30 18:20:08+00:00,270.0,43.28,43.25,46.00,0.00,0.000000,2.0,2,0.494390,True,REGULAR,USD,call,2025-11-28,0.057534,0.0191,JPM,314.209991
7,JPM251128C00275000,2025-10-29 15:03:05+00:00,275.0,33.00,39.15,40.75,0.00,0.000000,1.0,6,0.430914,True,REGULAR,USD,call,2025-11-28,0.057534,0.0191,JPM,314.209991
8,JPM251128C00280000,2025-11-03 18:24:42+00:00,280.0,31.15,34.30,35.95,0.00,0.000000,2.0,4,0.401129,True,REGULAR,USD,call,2025-11-28,0.057534,0.0191,JPM,314.209991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
879,JPM280121C00430000,2025-10-21 19:58:31+00:00,430.0,9.85,13.70,15.10,0.00,0.000000,1.0,2,0.250496,False,REGULAR,USD,call,2028-01-21,2.205479,0.0191,JPM,314.209991
880,JPM280121C00440000,2025-10-31 16:59:23+00:00,440.0,11.60,10.65,14.45,0.00,0.000000,4.0,14,0.255989,False,REGULAR,USD,call,2028-01-21,2.205479,0.0191,JPM,314.209991
881,JPM280121C00450000,2025-10-30 17:44:38+00:00,450.0,9.90,11.00,12.90,0.00,0.000000,4.0,17,0.254539,False,REGULAR,USD,call,2028-01-21,2.205479,0.0191,JPM,314.209991
882,JPM280121C00460000,2025-10-22 16:30:37+00:00,460.0,6.03,9.95,11.55,0.00,0.000000,2.0,6,0.253532,False,REGULAR,USD,call,2028-01-21,2.205479,0.0191,JPM,314.209991


In [228]:
calls_clean.describe()

,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,TTM,dividendYield,spot_price
count,837.000000,837.000000,837.000000,837.000000,837.000000,837.000000,837.000000,837.000000,837.000000,837.000000,8.370000e+02,837.000000
mean,280.400239,59.668220,61.124504,63.278292,0.010048,1.378497,65.540024,460.659498,0.428916,0.644006,1.910000e-02,314.209991
std,93.288559,59.706082,61.695467,62.784608,1.318865,35.732882,362.538300,1015.745730,0.388136,0.653501,3.471521e-18,0.000000
min,65.000000,0.010000,0.000000,0.000000,-5.450005,-98.461540,1.000000,0.000000,0.000010,0.000000,1.910000e-02,314.209991
25%,205.000000,7.000000,6.500000,7.800000,0.000000,0.000000,2.000000,18.000000,0.259926,0.115068,1.910000e-02,314.209991
50%,290.000000,37.330000,37.250000,39.700000,0.000000,0.000000,4.000000,82.000000,0.323218,0.364384,1.910000e-02,314.209991
75%,350.000000,107.010000,110.550000,113.300000,0.000000,0.000000,20.000000,384.000000,0.468999,1.112329,1.910000e-02,314.209991
max,470.000000,240.200000,242.750000,246.100000,13.110001,800.000240,6744.000000,8789.000000,6.646486,2.205479,1.910000e-02,314.209991


In [229]:
puts_clean = initial_dataset_clean(puts)
puts_clean

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,option_type,expiration,TTM,dividendYield,ticker,spot_price
1,JPM251128P00220000,2025-10-17 14:28:32+00:00,220.0,0.39,0.00,2.14,0.000000,0.000000,3.0,2,0.864747,False,REGULAR,USD,put,2025-11-28,0.057534,0.0191,JPM,314.209991
3,JPM251128P00240000,2025-10-17 19:24:44+00:00,240.0,0.60,0.00,2.17,0.000000,0.000000,10.0,11,0.687747,False,REGULAR,USD,put,2025-11-28,0.057534,0.0191,JPM,314.209991
4,JPM251128P00245000,2025-10-27 19:48:25+00:00,245.0,0.29,0.00,2.06,0.000000,0.000000,10.0,82,0.637211,False,REGULAR,USD,put,2025-11-28,0.057534,0.0191,JPM,314.209991
5,JPM251128P00250000,2025-10-24 15:22:43+00:00,250.0,0.48,0.00,2.22,0.000000,0.000000,2.0,3,0.604740,False,REGULAR,USD,put,2025-11-28,0.057534,0.0191,JPM,314.209991
6,JPM251128P00255000,2025-10-28 19:46:32+00:00,255.0,0.41,0.00,1.32,0.000000,0.000000,27.0,66,0.504155,False,REGULAR,USD,put,2025-11-28,0.057534,0.0191,JPM,314.209991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
774,JPM280121P00360000,2025-11-07 16:57:38+00:00,360.0,68.12,64.15,67.70,-0.649994,-0.945171,1.0,16,0.205490,True,REGULAR,USD,put,2028-01-21,2.205479,0.0191,JPM,314.209991
775,JPM280121P00370000,2025-11-07 17:20:14+00:00,370.0,75.25,70.00,74.50,-3.919998,-4.951368,11.0,3,0.202050,True,REGULAR,USD,put,2028-01-21,2.205479,0.0191,JPM,314.209991
776,JPM280121P00380000,2025-10-09 18:48:03+00:00,380.0,83.30,77.95,80.25,0.000000,0.000000,14.0,8,0.190171,True,REGULAR,USD,put,2028-01-21,2.205479,0.0191,JPM,314.209991
777,JPM280121P00390000,2025-11-07 19:10:43+00:00,390.0,88.60,84.95,86.95,-5.419998,-5.764729,10.0,4,0.181267,True,REGULAR,USD,put,2028-01-21,2.205479,0.0191,JPM,314.209991


**Join Calls and Puts**

In [230]:
df = pd.concat([puts_clean, calls_clean], ignore_index=True)
df

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,option_type,expiration,TTM,dividendYield,ticker,spot_price
0,JPM251128P00220000,2025-10-17 14:28:32+00:00,220.0,0.39,0.00,2.14,0.00,0.000000,3.0,2,0.864747,False,REGULAR,USD,put,2025-11-28,0.057534,0.0191,JPM,314.209991
1,JPM251128P00240000,2025-10-17 19:24:44+00:00,240.0,0.60,0.00,2.17,0.00,0.000000,10.0,11,0.687747,False,REGULAR,USD,put,2025-11-28,0.057534,0.0191,JPM,314.209991
2,JPM251128P00245000,2025-10-27 19:48:25+00:00,245.0,0.29,0.00,2.06,0.00,0.000000,10.0,82,0.637211,False,REGULAR,USD,put,2025-11-28,0.057534,0.0191,JPM,314.209991
3,JPM251128P00250000,2025-10-24 15:22:43+00:00,250.0,0.48,0.00,2.22,0.00,0.000000,2.0,3,0.604740,False,REGULAR,USD,put,2025-11-28,0.057534,0.0191,JPM,314.209991
4,JPM251128P00255000,2025-10-28 19:46:32+00:00,255.0,0.41,0.00,1.32,0.00,0.000000,27.0,66,0.504155,False,REGULAR,USD,put,2025-11-28,0.057534,0.0191,JPM,314.209991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1570,JPM280121C00430000,2025-10-21 19:58:31+00:00,430.0,9.85,13.70,15.10,0.00,0.000000,1.0,2,0.250496,False,REGULAR,USD,call,2028-01-21,2.205479,0.0191,JPM,314.209991
1571,JPM280121C00440000,2025-10-31 16:59:23+00:00,440.0,11.60,10.65,14.45,0.00,0.000000,4.0,14,0.255989,False,REGULAR,USD,call,2028-01-21,2.205479,0.0191,JPM,314.209991
1572,JPM280121C00450000,2025-10-30 17:44:38+00:00,450.0,9.90,11.00,12.90,0.00,0.000000,4.0,17,0.254539,False,REGULAR,USD,call,2028-01-21,2.205479,0.0191,JPM,314.209991
1573,JPM280121C00460000,2025-10-22 16:30:37+00:00,460.0,6.03,9.95,11.55,0.00,0.000000,2.0,6,0.253532,False,REGULAR,USD,call,2028-01-21,2.205479,0.0191,JPM,314.209991


In [231]:
def optionType(OPTtype):
    # Puts are type 1
    if OPTtype == "put":
        return "put"
    else:
    # Calls are type 0
        return "call"

In [232]:
df["optionType"] = df["option_type"].apply(optionType)
#df.drop(columns="option_type", inplace=True)
df

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,...,inTheMoney,contractSize,currency,option_type,expiration,TTM,dividendYield,ticker,spot_price,optionType
0,JPM251128P00220000,2025-10-17 14:28:32+00:00,220.0,0.39,0.00,2.14,0.00,0.000000,3.0,2,...,False,REGULAR,USD,put,2025-11-28,0.057534,0.0191,JPM,314.209991,put
1,JPM251128P00240000,2025-10-17 19:24:44+00:00,240.0,0.60,0.00,2.17,0.00,0.000000,10.0,11,...,False,REGULAR,USD,put,2025-11-28,0.057534,0.0191,JPM,314.209991,put
2,JPM251128P00245000,2025-10-27 19:48:25+00:00,245.0,0.29,0.00,2.06,0.00,0.000000,10.0,82,...,False,REGULAR,USD,put,2025-11-28,0.057534,0.0191,JPM,314.209991,put
3,JPM251128P00250000,2025-10-24 15:22:43+00:00,250.0,0.48,0.00,2.22,0.00,0.000000,2.0,3,...,False,REGULAR,USD,put,2025-11-28,0.057534,0.0191,JPM,314.209991,put
4,JPM251128P00255000,2025-10-28 19:46:32+00:00,255.0,0.41,0.00,1.32,0.00,0.000000,27.0,66,...,False,REGULAR,USD,put,2025-11-28,0.057534,0.0191,JPM,314.209991,put
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1570,JPM280121C00430000,2025-10-21 19:58:31+00:00,430.0,9.85,13.70,15.10,0.00,0.000000,1.0,2,...,False,REGULAR,USD,call,2028-01-21,2.205479,0.0191,JPM,314.209991,call
1571,JPM280121C00440000,2025-10-31 16:59:23+00:00,440.0,11.60,10.65,14.45,0.00,0.000000,4.0,14,...,False,REGULAR,USD,call,2028-01-21,2.205479,0.0191,JPM,314.209991,call
1572,JPM280121C00450000,2025-10-30 17:44:38+00:00,450.0,9.90,11.00,12.90,0.00,0.000000,4.0,17,...,False,REGULAR,USD,call,2028-01-21,2.205479,0.0191,JPM,314.209991,call
1573,JPM280121C00460000,2025-10-22 16:30:37+00:00,460.0,6.03,9.95,11.55,0.00,0.000000,2.0,6,...,False,REGULAR,USD,call,2028-01-21,2.205479,0.0191,JPM,314.209991,call


**Interest Rate Interpolation**

In [233]:
# Needs to figure out if we're using a interpolation structure to match each of them, also considering zero bond rates for the interpolation
def interest_rate(row):
    # Need to change this to interpolation calculation
    return (0.04)

In [234]:
df["r"] = .04
df

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,...,contractSize,currency,option_type,expiration,TTM,dividendYield,ticker,spot_price,optionType,r
0,JPM251128P00220000,2025-10-17 14:28:32+00:00,220.0,0.39,0.00,2.14,0.00,0.000000,3.0,2,...,REGULAR,USD,put,2025-11-28,0.057534,0.0191,JPM,314.209991,put,0.04
1,JPM251128P00240000,2025-10-17 19:24:44+00:00,240.0,0.60,0.00,2.17,0.00,0.000000,10.0,11,...,REGULAR,USD,put,2025-11-28,0.057534,0.0191,JPM,314.209991,put,0.04
2,JPM251128P00245000,2025-10-27 19:48:25+00:00,245.0,0.29,0.00,2.06,0.00,0.000000,10.0,82,...,REGULAR,USD,put,2025-11-28,0.057534,0.0191,JPM,314.209991,put,0.04
3,JPM251128P00250000,2025-10-24 15:22:43+00:00,250.0,0.48,0.00,2.22,0.00,0.000000,2.0,3,...,REGULAR,USD,put,2025-11-28,0.057534,0.0191,JPM,314.209991,put,0.04
4,JPM251128P00255000,2025-10-28 19:46:32+00:00,255.0,0.41,0.00,1.32,0.00,0.000000,27.0,66,...,REGULAR,USD,put,2025-11-28,0.057534,0.0191,JPM,314.209991,put,0.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1570,JPM280121C00430000,2025-10-21 19:58:31+00:00,430.0,9.85,13.70,15.10,0.00,0.000000,1.0,2,...,REGULAR,USD,call,2028-01-21,2.205479,0.0191,JPM,314.209991,call,0.04
1571,JPM280121C00440000,2025-10-31 16:59:23+00:00,440.0,11.60,10.65,14.45,0.00,0.000000,4.0,14,...,REGULAR,USD,call,2028-01-21,2.205479,0.0191,JPM,314.209991,call,0.04
1572,JPM280121C00450000,2025-10-30 17:44:38+00:00,450.0,9.90,11.00,12.90,0.00,0.000000,4.0,17,...,REGULAR,USD,call,2028-01-21,2.205479,0.0191,JPM,314.209991,call,0.04
1573,JPM280121C00460000,2025-10-22 16:30:37+00:00,460.0,6.03,9.95,11.55,0.00,0.000000,2.0,6,...,REGULAR,USD,call,2028-01-21,2.205479,0.0191,JPM,314.209991,call,0.04


**Forward log-Moneyness**

We decided to use forward log-moneyness because its adjusted for r and q which are crucial in pricing the EEP

In [235]:

def forward_log_moneyness(S: float, r:float, q:float, T:float, K:float):
    """
    Calculates the log_moneyness in relation to the Forward Price of the stock

    Parameters
    ----------
    S : float
        Current spot_price of the stock.
    r : float
        Risk-free rate associated with that particular option maturity
    q : float
        Dividend-yield rate associated with that particular option maturity
    T : float
        Maturity of the option contract
    K : float
        Strike price of the option contract

    Returns
    -------
    Float
        - Forward log moneyness 
    """
    forward_price = S * np.exp((r - q)*T)
    
    return np.log(K/forward_price)

In [236]:
df["forward_log_moneyness"] = forward_log_moneyness(
    S=df["spot_price"].to_numpy(),
    r=df["r"].to_numpy(),
    q=df["dividendYield"].to_numpy(),
    T=df["TTM"].to_numpy(),
    K=df["strike"].to_numpy(),
)

df

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,...,currency,option_type,expiration,TTM,dividendYield,ticker,spot_price,optionType,r,forward_log_moneyness
0,JPM251128P00220000,2025-10-17 14:28:32+00:00,220.0,0.39,0.00,2.14,0.00,0.000000,3.0,2,...,USD,put,2025-11-28,0.057534,0.0191,JPM,314.209991,put,0.04,-0.357636
1,JPM251128P00240000,2025-10-17 19:24:44+00:00,240.0,0.60,0.00,2.17,0.00,0.000000,10.0,11,...,USD,put,2025-11-28,0.057534,0.0191,JPM,314.209991,put,0.04,-0.270625
2,JPM251128P00245000,2025-10-27 19:48:25+00:00,245.0,0.29,0.00,2.06,0.00,0.000000,10.0,82,...,USD,put,2025-11-28,0.057534,0.0191,JPM,314.209991,put,0.04,-0.250006
3,JPM251128P00250000,2025-10-24 15:22:43+00:00,250.0,0.48,0.00,2.22,0.00,0.000000,2.0,3,...,USD,put,2025-11-28,0.057534,0.0191,JPM,314.209991,put,0.04,-0.229803
4,JPM251128P00255000,2025-10-28 19:46:32+00:00,255.0,0.41,0.00,1.32,0.00,0.000000,27.0,66,...,USD,put,2025-11-28,0.057534,0.0191,JPM,314.209991,put,0.04,-0.210000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1570,JPM280121C00430000,2025-10-21 19:58:31+00:00,430.0,9.85,13.70,15.10,0.00,0.000000,1.0,2,...,USD,call,2028-01-21,2.205479,0.0191,JPM,314.209991,call,0.04,0.267629
1571,JPM280121C00440000,2025-10-31 16:59:23+00:00,440.0,11.60,10.65,14.45,0.00,0.000000,4.0,14,...,USD,call,2028-01-21,2.205479,0.0191,JPM,314.209991,call,0.04,0.290619
1572,JPM280121C00450000,2025-10-30 17:44:38+00:00,450.0,9.90,11.00,12.90,0.00,0.000000,4.0,17,...,USD,call,2028-01-21,2.205479,0.0191,JPM,314.209991,call,0.04,0.313092
1573,JPM280121C00460000,2025-10-22 16:30:37+00:00,460.0,6.03,9.95,11.55,0.00,0.000000,2.0,6,...,USD,call,2028-01-21,2.205479,0.0191,JPM,314.209991,call,0.04,0.335070


Calculate Mid Price

In [237]:
def mid_price(bid: float, ask: float):
    """
    Calcualtes the mid-price of the option based on bid and ask prices

    Parameters
    ----------
    bid : float
        Current bid price of the option
    ask : float
        Current ask price of the option

    Returns
    -------
    Float
        - Mid price
    """

    return ((bid+ask)/2)
    

In [238]:
df["midPrice"] = mid_price(df["bid"], df["ask"])
df

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,...,option_type,expiration,TTM,dividendYield,ticker,spot_price,optionType,r,forward_log_moneyness,midPrice
0,JPM251128P00220000,2025-10-17 14:28:32+00:00,220.0,0.39,0.00,2.14,0.00,0.000000,3.0,2,...,put,2025-11-28,0.057534,0.0191,JPM,314.209991,put,0.04,-0.357636,1.070
1,JPM251128P00240000,2025-10-17 19:24:44+00:00,240.0,0.60,0.00,2.17,0.00,0.000000,10.0,11,...,put,2025-11-28,0.057534,0.0191,JPM,314.209991,put,0.04,-0.270625,1.085
2,JPM251128P00245000,2025-10-27 19:48:25+00:00,245.0,0.29,0.00,2.06,0.00,0.000000,10.0,82,...,put,2025-11-28,0.057534,0.0191,JPM,314.209991,put,0.04,-0.250006,1.030
3,JPM251128P00250000,2025-10-24 15:22:43+00:00,250.0,0.48,0.00,2.22,0.00,0.000000,2.0,3,...,put,2025-11-28,0.057534,0.0191,JPM,314.209991,put,0.04,-0.229803,1.110
4,JPM251128P00255000,2025-10-28 19:46:32+00:00,255.0,0.41,0.00,1.32,0.00,0.000000,27.0,66,...,put,2025-11-28,0.057534,0.0191,JPM,314.209991,put,0.04,-0.210000,0.660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1570,JPM280121C00430000,2025-10-21 19:58:31+00:00,430.0,9.85,13.70,15.10,0.00,0.000000,1.0,2,...,call,2028-01-21,2.205479,0.0191,JPM,314.209991,call,0.04,0.267629,14.400
1571,JPM280121C00440000,2025-10-31 16:59:23+00:00,440.0,11.60,10.65,14.45,0.00,0.000000,4.0,14,...,call,2028-01-21,2.205479,0.0191,JPM,314.209991,call,0.04,0.290619,12.550
1572,JPM280121C00450000,2025-10-30 17:44:38+00:00,450.0,9.90,11.00,12.90,0.00,0.000000,4.0,17,...,call,2028-01-21,2.205479,0.0191,JPM,314.209991,call,0.04,0.313092,11.950
1573,JPM280121C00460000,2025-10-22 16:30:37+00:00,460.0,6.03,9.95,11.55,0.00,0.000000,2.0,6,...,call,2028-01-21,2.205479,0.0191,JPM,314.209991,call,0.04,0.335070,10.750


In [239]:
from heston import calibrate_and_price_heston_european

V_eu = calibrate_and_price_heston_european(df, group_cols=("ticker", ))
df = df.join(V_eu)
df

[deAm] === group start === :: group=('JPM',) | rows=1575
[deAm] initial bracket :: tree=jr | steps=400 | lo=0.001 | f_lo=-1.07 | hi=6.0 | f_hi=96.07828042325012 | P=1.07
[deAm] sigma* :: tree=jr | sigma_star=0.8890845367284084
[deAm] eu price :: tree=jr | p_eu=1.0778159756783583
[deAm] IV: QL solver :: iv=0.8890845241561225 | price_for_iv=1.0778159756783583
[deAm] helper IV :: idx=0 | iv=0.8890845241561225
[deAm] initial bracket :: tree=jr | steps=400 | lo=0.001 | f_lo=-1.085 | hi=6.0 | f_hi=109.87016879272952 | P=1.085
[deAm] sigma* :: tree=jr | sigma_star=0.7072266264906322
[deAm] eu price :: tree=jr | p_eu=1.0899722763137807
[deAm] IV: QL solver :: iv=0.707226626498426 | price_for_iv=1.0899722763137807
[deAm] helper IV :: idx=1 | iv=0.707226626498426
[deAm] initial bracket :: tree=jr | steps=400 | lo=0.001 | f_lo=-1.03 | hi=6.0 | f_hi=113.37722221136765 | P=1.03
[deAm] sigma* :: tree=jr | sigma_star=0.6556204632725944
[deAm] eu price :: tree=jr | p_eu=1.0346950358565448
[deAm] IV: Q

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,...,expiration,TTM,dividendYield,ticker,spot_price,optionType,r,forward_log_moneyness,midPrice,V_EU_Heston
0,JPM251128P00220000,2025-10-17 14:28:32+00:00,220.0,0.39,0.00,2.14,0.00,0.000000,3.0,2,...,2025-11-28,0.057534,0.0191,JPM,314.209991,put,0.04,-0.357636,1.070,0.026537
1,JPM251128P00240000,2025-10-17 19:24:44+00:00,240.0,0.60,0.00,2.17,0.00,0.000000,10.0,11,...,2025-11-28,0.057534,0.0191,JPM,314.209991,put,0.04,-0.270625,1.085,0.088794
2,JPM251128P00245000,2025-10-27 19:48:25+00:00,245.0,0.29,0.00,2.06,0.00,0.000000,10.0,82,...,2025-11-28,0.057534,0.0191,JPM,314.209991,put,0.04,-0.250006,1.030,0.118372
3,JPM251128P00250000,2025-10-24 15:22:43+00:00,250.0,0.48,0.00,2.22,0.00,0.000000,2.0,3,...,2025-11-28,0.057534,0.0191,JPM,314.209991,put,0.04,-0.229803,1.110,0.157003
4,JPM251128P00255000,2025-10-28 19:46:32+00:00,255.0,0.41,0.00,1.32,0.00,0.000000,27.0,66,...,2025-11-28,0.057534,0.0191,JPM,314.209991,put,0.04,-0.210000,0.660,0.207268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1570,JPM280121C00430000,2025-10-21 19:58:31+00:00,430.0,9.85,13.70,15.10,0.00,0.000000,1.0,2,...,2028-01-21,2.205479,0.0191,JPM,314.209991,call,0.04,0.267629,14.400,17.507524
1571,JPM280121C00440000,2025-10-31 16:59:23+00:00,440.0,11.60,10.65,14.45,0.00,0.000000,4.0,14,...,2028-01-21,2.205479,0.0191,JPM,314.209991,call,0.04,0.290619,12.550,15.743792
1572,JPM280121C00450000,2025-10-30 17:44:38+00:00,450.0,9.90,11.00,12.90,0.00,0.000000,4.0,17,...,2028-01-21,2.205479,0.0191,JPM,314.209991,call,0.04,0.313092,11.950,14.151324
1573,JPM280121C00460000,2025-10-22 16:30:37+00:00,460.0,6.03,9.95,11.55,0.00,0.000000,2.0,6,...,2028-01-21,2.205479,0.0191,JPM,314.209991,call,0.04,0.335070,10.750,12.716312


In [240]:
df.describe()

,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,TTM,dividendYield,spot_price,r,forward_log_moneyness,midPrice,V_EU_Heston
count,1575.000000,1575.000000,1575.000000,1575.000000,1575.000000,1575.000000,1575.000000,1575.000000,1575.000000,1575.000000,1.575000e+03,1575.000000,1.575000e+03,1575.000000,1575.000000,1.575000e+03
mean,262.500000,37.428844,37.450444,39.068203,-0.022857,9.457881,56.989206,488.363175,0.469281,0.627869,1.910000e-02,314.209991,4.000000e-02,-0.257373,38.259324,3.892001e+01
std,87.490288,51.703350,53.208820,54.254441,1.104338,182.361424,290.838084,969.676568,0.480324,0.654474,3.470549e-18,0.000000,6.941098e-18,0.377912,53.730146,5.576793e+01
min,65.000000,0.010000,0.000000,0.000000,-11.660000,-98.461540,1.000000,0.000000,0.000010,0.000000,1.910000e-02,314.209991,4.000000e-02,-1.578079,0.000000,-5.041858e-12
25%,190.000000,1.285000,0.920000,1.965000,0.000000,0.000000,2.000000,20.000000,0.259910,0.115068,1.910000e-02,314.209991,4.000000e-02,-0.510663,1.172500,1.195223e+00
50%,270.000000,10.950000,10.450000,11.550000,0.000000,0.000000,5.000000,102.000000,0.329841,0.364384,1.910000e-02,314.209991,4.000000e-02,-0.161366,10.775000,1.050494e+01
75%,321.250000,54.325000,52.700000,54.750000,0.000000,0.000000,22.000000,488.000000,0.488958,0.987671,1.910000e-02,314.209991,4.000000e-02,0.017458,53.725000,5.426174e+01
max,470.000000,240.200000,242.750000,246.100000,13.110001,4800.004400,6744.000000,8789.000000,6.646486,2.205479,1.910000e-02,314.209991,4.000000e-02,0.384634,244.425000,2.488187e+02


In [241]:
df.columns

Index(['contractSymbol', 'lastTradeDate', 'strike', 'lastPrice', 'bid', 'ask',
       'change', 'percentChange', 'volume', 'openInterest',
       'impliedVolatility', 'inTheMoney', 'contractSize', 'currency',
       'option_type', 'expiration', 'TTM', 'dividendYield', 'ticker',
       'spot_price', 'optionType', 'r', 'forward_log_moneyness', 'midPrice',
       'V_EU_Heston'],
      dtype='object')